This file is for creating the dataset that contains a new entry every 6 visits per patient. We also run the XGBoost model with hyperparameter tuning. 

In [1]:
import numpy as np
import os
import pandas as pd
import pyodbc
import time
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from datetime import datetime
import sys
import gc
from scipy.sparse import csr_matrix
import pickle
import joblib
from itertools import product
import matplotlib



from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [2]:
connection_string = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=OMOP.DBMI.COLUMBIA.EDU;'
    'DATABASE=cdm_mdcd;'
    'TRUSTED_CONNECTION=YES;')

conn = pyodbc.connect(connection_string)

In [3]:
path = '../'

# DATASET CREATION

### Import the population dataframe and constrict to the correct set of patients

In [ ]:
num_days_prediction = 90
df_pop = pd.read_csv(path+'population.csv')
df_pop.rename({'psychosis_dx_date':'psychosis_diagnosis_date'}, axis=1, inplace=True)
df_pop['psychosis_diagnosis_date'] = pd.to_datetime(df_pop['psychosis_diagnosis_date'], format="mixed")
df_pop['cohort_start_date'] = pd.to_datetime(df_pop['cohort_start_date'])
df_pop = df_pop.loc[(df_pop['cohort_start_date']-df_pop['psychosis_diagnosis_date']).dt.days >= num_days_prediction]

In [ ]:
all_visits = pd.read_csv(path+'temporal_visits.csv')
df_pop = df_pop.merge(all_visits.groupby('person_id').min()['visit_start_date'], how='left', left_on='person_id',right_index=True)
df_pop.rename({'visit_start_date':'first_visit'}, axis=1, inplace=True)
df_pop.head()

### Import the temporal data

In [ ]:
all_conds = pd.read_csv(path+'temporal_conditions.csv')
all_meds = pd.read_csv(path+'temporal_medications.csv')
all_procedures = pd.read_csv(path+'temporal_procedures.csv')
all_labs = pd.read_csv(path+'temporal_labs.csv')

### Restrict to appropriate time periods

In [ ]:
all_meds = all_meds.loc[all_meds['person_id'].isin(df_pop['person_id'])]
all_meds['cohort_start_date'] = pd.to_datetime(all_meds['cohort_start_date'])
all_meds['drug_era_start_date'] = pd.to_datetime(all_meds['drug_era_start_date'])
all_meds['drug_era_end_date'] = pd.to_datetime(all_meds['drug_era_end_date'])
all_meds = all_meds.loc[(all_meds['cohort_start_date']-all_meds['drug_era_end_date']).dt.days >= num_days_prediction]
all_meds['days_to_cohort_start'] = (all_meds['cohort_start_date']-all_meds['drug_era_start_date']).dt.days

In [ ]:
all_visits = all_visits.loc[all_visits['person_id'].isin(df_pop['person_id'])]
all_visits['cohort_start_date'] = pd.to_datetime(all_visits['cohort_start_date'])
all_visits['visit_start_date'] = pd.to_datetime(all_visits['visit_start_date'])
all_visits['visit_end_date'] = pd.to_datetime(all_visits['visit_end_date'])
all_visits = all_visits.loc[(all_visits['cohort_start_date']-all_visits['visit_end_date']).dt.days >= num_days_prediction]
all_visits['days_to_cohort_start'] = (all_visits['cohort_start_date']-all_visits['visit_start_date']).dt.days

In [ ]:
all_conds = all_conds.loc[all_conds['person_id'].isin(df_pop['person_id'])]
all_conds['cohort_start_date'] = pd.to_datetime(all_conds['cohort_start_date'])
all_conds['condition_start_date'] = pd.to_datetime(all_conds['condition_start_date'])
all_conds['days_to_cohort_start'] = (all_conds['cohort_start_date']-all_conds['condition_start_date']).dt.days
all_conds = all_conds.loc[all_conds['days_to_cohort_start'] >= num_days_prediction]

In [ ]:
all_procedures = all_procedures.loc[all_procedures['person_id'].isin(df_pop['person_id'])]
all_procedures['cohort_start_date'] = pd.to_datetime(all_procedures['cohort_start_date'])
all_procedures['procedure_date'] = pd.to_datetime(all_procedures['procedure_date'])
all_procedures['days_to_cohort_start'] = (all_procedures['cohort_start_date']-all_procedures['procedure_date']).dt.days
all_procedures = all_procedures.loc[all_procedures['days_to_cohort_start'] >= num_days_prediction]

In [ ]:
all_labs = all_labs.loc[all_labs['person_id'].isin(df_pop['person_id'])]
all_labs['cohort_start_date'] = pd.to_datetime(all_labs['cohort_start_date'])
all_labs['measurement_date'] = pd.to_datetime(all_labs['measurement_date'])
all_labs['days_to_cohort_start'] = (all_labs['cohort_start_date']-all_labs['measurement_date']).dt.days
all_labs = all_labs.loc[all_labs['days_to_cohort_start'] >= num_days_prediction]

In [ ]:
all_labs['concept_name'].replace({'Methadone':'Methadone_Lab'}, inplace=True)
all_procedures['concept_name'].replace({'Methadone':'Methadone_Procedure'}, inplace=True)

### delete rare occurrences (e.g. any concept id that does not appear at least once for unique patients)

In [ ]:
def drop_rare_occurrences(df, col_concept, col_id = 'person_id', size_pop = len(df_pop)):
    unique_occurrences = df[['person_id', col_concept]].drop_duplicates()
    unique_occurrences = unique_occurrences.value_counts(col_concept)
    common_occurrences = unique_occurrences[unique_occurrences/size_pop > 0.01].index
    return df.loc[df[col_concept].isin(common_occurrences)]
all_conds = drop_rare_occurrences(all_conds, 'condition_concept_id')
all_meds = drop_rare_occurrences(all_meds, 'drug_concept_id')
all_procedures = drop_rare_occurrences(all_procedures, 'procedure_concept_id')
all_labs = drop_rare_occurrences(all_labs, 'measurement_concept_id')

### Check that the minimum time between cohort start date and start/end dates for healthcare services is over 90 days

In [ ]:
check = (all_labs['cohort_start_date']-all_labs['measurement_date']).dt.days
print('Labs:', check.min(), check.max())

check = (all_procedures['cohort_start_date']-all_procedures['procedure_date']).dt.days
print('Procedures:', check.min(), check.max())

check = (all_conds['cohort_start_date']-all_conds['condition_start_date']).dt.days
print('Conditions:', check.min(), check.max())

check = (all_meds['cohort_start_date']-all_meds['drug_era_start_date']).dt.days
print('Meds (Start of prescription):', check.min(), check.max())
check = (all_meds['cohort_start_date']-all_meds['drug_era_end_date']).dt.days
print('Meds (End of prescription):', check.min(), check.max())

check = (all_visits['cohort_start_date']-all_visits['visit_start_date']).dt.days
print('Visits (Start of visit):', check.min(), check.max())
check = (all_visits['cohort_start_date']-all_visits['visit_end_date']).dt.days
print('Visits (End of visit):', check.min(), check.max())

print('Check presence of SCZ:',len(all_conds.loc[all_conds['concept_name'].isin(['Schizophrenia', 'Paranoid schizophrenia'])]))

### check that the cohort start date is the same across all dfs

In [ ]:
check_cohort_start = df_pop[['person_id','cohort_start_date']]
check_cohort_start = check_cohort_start.merge(all_conds[['person_id','cohort_start_date']].drop_duplicates(),how='left', left_on='person_id', right_on='person_id', suffixes=['_pop','_cond'])
check_cohort_start = check_cohort_start.merge(all_visits[['person_id','cohort_start_date']].drop_duplicates(),how='left', left_on='person_id', right_on='person_id', suffixes = ['_old1','_visits'])
check_cohort_start = check_cohort_start.merge(all_procedures[['person_id','cohort_start_date']].drop_duplicates(),how='left', left_on='person_id', right_on='person_id', suffixes=['_old2','_pro'])
check_cohort_start = check_cohort_start.merge(all_labs[['person_id','cohort_start_date']].drop_duplicates(),how='left', left_on='person_id', right_on='person_id', suffixes=['_old3','_labs'])
check_cohort_start = check_cohort_start.merge(all_meds[['person_id','cohort_start_date']].drop_duplicates(),how='left', left_on='person_id', right_on='person_id', suffixes=['_old4','_meds'])
check_cohort_start.set_index('person_id',inplace=True)
check_cohort_start = check_cohort_start.T
num_unique = check_cohort_start.T.apply(lambda x: x.nunique(), axis=1)
print('Number of places where cohort start date doesnt align:',(num_unique>1).sum())

### Make SQL queries for grouping conditions and medications

In [ ]:
# conditions mapping
conditions_mapping_query = ("SELECT c_icd10.concept_id as icd10_ancestor_concept_id,c_icd10.concept_name as icd10_ancestor_concept_name, c_icd10.concept_code as icd10_code, rel.concept_id_2 as standard_ancestor_concept_id, c_rel.concept_name as standard_ancestor_concept_name, ca.descendant_concept_id as standard_descendant_concept_id, c_new.concept_name as standard_descendant_concept_name, c_new.standard_concept as check_standard "+
    "FROM dbo.concept as c_icd10 "+
    "LEFT JOIN dbo.concept_relationship as rel on rel.concept_id_1 = c_icd10.concept_id "+
    "LEFT JOIN dbo.concept as c_rel on rel.concept_id_2 = c_rel.concept_id "+
    "LEFT JOIN dbo.concept_ancestor as ca ON ca.ancestor_concept_id = rel.concept_id_2 "+
    "LEFT JOIN dbo.concept as c_new on c_new.concept_id = ca.descendant_concept_id "+
    "WHERE c_icd10.concept_class_id = '3-char nonbill code' and c_icd10.vocabulary_id = 'ICD10CM' "+
    "AND rel.relationship_id = 'Maps to' AND c_rel.standard_concept = 'S'")
conditions_mapping = pd.io.sql.read_sql(conditions_mapping_query, conn)

conditions_mapping = conditions_mapping.loc[conditions_mapping['standard_descendant_concept_id'].isin(list(all_conds['condition_concept_id'].unique()))]

In [ ]:
# medications mapping 
medications_mapping_query = ("SELECT c_atc.concept_id as atc_concept_id, c_atc.concept_name as atc_concept_name, c_standard.concept_id as standard_concept_id, c_standard.concept_name as standard_concept_name "+
                             "FROM dbo.concept as c_atc "+
                             "LEFT JOIN dbo.concept_ancestor as ca on ancestor_concept_id=c_atc.concept_id "+
                             "LEFT JOIN dbo.concept as c_standard on c_standard.concept_id = descendant_concept_id "+
                             "WHERE c_atc.concept_class_id = 'ATC 3rd' AND c_standard.standard_concept = 'S'")

medications_mapping = pd.io.sql.read_sql(medications_mapping_query, conn)
medications_mapping = medications_mapping.loc[medications_mapping['standard_concept_id'].isin(list(all_meds['drug_concept_id'].unique()))]

In [ ]:
# medications mapping: move Lithium to the antiepileptics category
def generate_code_list(drugtype, concept_class):
    sql_query = ("SELECT ancestor_concept_id, descendant_concept_id, concept_name " + 
               "FROM dbo.concept_ancestor JOIN dbo.concept ON descendant_concept_id = concept_id "+
               "WHERE ancestor_concept_id = (SELECT concept_id from dbo.concept WHERE concept_class_id = '"+concept_class+"' AND concept_name = '"+drugtype+"');")
    codes_list = pd.read_sql(sql_query, conn)
    return list(codes_list['descendant_concept_id'])

lithium_list = generate_code_list('Lithium', 'ATC 4th')
medications_mapping.loc[(medications_mapping['standard_concept_id'].isin(lithium_list))&(medications_mapping['atc_concept_name']=='ANTIPSYCHOTICS'), 'atc_concept_name'] = 'ANTIEPILEPTICS'
medications_mapping['atc_concept_name'].replace({'ANTIEPILEPTICS': 'MOOD STABILIZERS'}, inplace=True)

In [ ]:
query = ("SELECT vo.person_id, vo.visit_occurrence_id, vo.visit_concept_id, co.condition_start_date, vo.visit_start_date, vo.visit_end_date, co.condition_concept_id, c.concept_name as condition_name, p.race_concept_id, p.gender_concept_id "+
         "FROM dbo.visit_occurrence as vo LEFT JOIN dbo.condition_occurrence as co on co.visit_occurrence_id = vo.visit_occurrence_id "+
         "LEFT JOIN dbo.concept as c on c.concept_id = co.condition_concept_id "+
         "LEFT JOIN dbo.person as p on p.person_id = vo.person_id "+
         "WHERE vo.visit_concept_id = 9201 AND condition_concept_id IN "+
         "(SELECT DISTINCT concept_id_2 FROM dbo.concept as c LEFT JOIN dbo.concept_relationship on concept_id_1 = concept_id WHERE c.concept_code LIKE 'F%' AND c.vocabulary_id = 'ICD10CM' AND relationship_id = 'Maps to')")

psych_hosp = pd.io.sql.read_sql(query, conn)
list_psych_visits = list(psych_hosp['visit_occurrence_id'].unique())

### Define a function that gives us a dataframe of person_id by features (count per year)

In [ ]:
# temp pop needs to have a "years obs" column
def make_static_df(temp_pop, temp_conds, temp_meds, temp_visits, temp_procedures, temp_labs):
    ### CONDITIONS
    count_conds = temp_conds.groupby(by=["person_id", "condition_concept_id"]).size().reset_index()
    cond_features = pd.DataFrame(data=0, index=temp_pop['person_id'], columns=conditions_mapping['icd10_code'].unique())
    icd_dict = conditions_mapping.groupby('icd10_code')['standard_descendant_concept_id'].apply(list).to_dict()

    list_icd_codes = list(conditions_mapping['icd10_code'].unique())
    for i in (range(0,len(icd_dict))):
        temp = count_conds.loc[count_conds['condition_concept_id'].isin(icd_dict[list_icd_codes[i]])].groupby('person_id').count()['condition_concept_id']
        cond_features.loc[temp.index, list_icd_codes[i]] = temp.values
        
    cond_features_binary = (cond_features > 0)*1
    # eliminate icd10 codes with <= 1% prevalence
    #cond_features_binary = cond_features_binary[cond_features_binary.columns[100*cond_features_binary.sum(axis=0)/len(cond_features_binary)>1]]
    #cond_features = cond_features[cond_features_binary.columns[100*cond_features_binary.sum(axis=0)/len(cond_features_binary)>1]]
    # adjust so that cond_features is per year of observation
    cond_features = cond_features.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    cond_features = cond_features.div(cond_features.years_obs, axis=0) 
    cond_features.drop(['years_obs'], axis=1, inplace=True)
    
    ### MEDICATIONS
    med_features = pd.DataFrame(data=0, index=temp_pop['person_id'], columns=medications_mapping['atc_concept_name'].unique())
    meds_dict = medications_mapping.groupby('atc_concept_name')['standard_concept_id'].apply(list).to_dict()
    temp_meds['drug_exposure_days'] = (temp_meds['drug_era_end_date']-temp_meds['drug_era_start_date']).dt.days
    count_meds = temp_meds[['person_id', 'drug_concept_id', 'drug_exposure_days']].groupby(['person_id', 'drug_concept_id']).sum().reset_index()

    list_atc_codes = list(medications_mapping['atc_concept_name'].unique())
    for i in (range(0,len(meds_dict))):
        temp = count_meds.loc[count_meds['drug_concept_id'].isin(meds_dict[list_atc_codes[i]])].groupby('person_id')['drug_exposure_days'].sum()
        med_features.loc[temp.index, list_atc_codes[i]] = temp.values
        
    # limit to only medication classes with > 1% prevalence
    #med_features_binary = (med_features > 0)*1
    #med_cols = list(med_features_binary.columns[100*med_features_binary.sum(axis=0)/len(med_features_binary)>1])
    #med_features = med_features[med_cols]
    # adjust so that med_features is per year of observation
    med_features = med_features.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    med_features = med_features.div(med_features.years_obs, axis=0) 
    med_features.drop(['years_obs'], axis=1, inplace=True)

    ###VISITS
    # Time from most recent visit to end of observation period
    temp_visits.merge(temp_pop[['person_id', 'cutoff_date']], how='left', left_on = 'person_id', right_on='person_id')
    temp_visits['days_to_end_obs'] = (temp_visits['cutoff_date']-temp_visits['visit_end_date']).dt.days
    if (temp_visits['visit_start_date'] > temp_visits['cutoff_date']).sum()>0:
        print('Start date issue')
    if (temp_visits['visit_end_date'] > temp_visits['cutoff_date']).sum()>0:
        print('End date issue')
    if (temp_visits['days_to_end_obs']).max() < 0:
        print('Issue with end obs')

    visits_timing = temp_visits.groupby(['person_id', 'visit_concept_id']).min()['days_to_end_obs']
    visits_timing = visits_timing.reset_index()
    visits_timing = visits_timing.pivot_table(index='person_id', columns = 'visit_concept_id', values='days_to_end_obs', fill_value = 2190)
    visits_timing.rename({9201:'most_recent_inpatient', 9202: 'most_recent_outpatient', 9203:'most_recent_ED', 42898160:'most_recent_nonhospital'}, axis=1, inplace=True)
    
    # Number of visits
    num_visits = temp_visits.groupby(['person_id', 'visit_concept_id']).count()['cohort_start_date'].reset_index()
    num_visits = num_visits.pivot_table(index='person_id', columns = 'visit_concept_id', values = 'cohort_start_date', fill_value=0)
    num_visits.rename({9201:'num_visits_inpatient', 9202: 'num_visits_outpatient', 9203:'num_visits_ED', 42898160:'num_visits_nonhospital'}, axis=1, inplace=True)
    # adjust so that num_visits is per year of observation
    num_visits = num_visits.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    num_visits = num_visits.div(num_visits.years_obs, axis=0) 
    num_visits.drop(['years_obs'], axis=1, inplace=True)
    
    # Length of Stay
    non_outpatient = temp_visits.loc[temp_visits['visit_concept_id']!=9202]

    non_outpatient['los'] = (non_outpatient['visit_end_date']-non_outpatient['visit_start_date']).dt.days
    los = non_outpatient.groupby(['person_id', 'visit_concept_id']).agg({'los':['sum', 'max', 'min', 'mean']})
    los = los.reset_index()
    los.columns = [' '.join(col).strip() for col in los.columns.values]

    los = los.pivot_table(index='person_id', columns = 'visit_concept_id', values=['los sum', 'los max', 'los min', 'los mean'], fill_value = 0)
    los.columns = [''.join(str(col)).strip() for col in los.columns.values]

    #rename columns
    if len(los.columns) == 12:
        los.columns = ['los_max_inpatient', 'los_max_ed', 'los_max_nonhospitalization',
        'los_mean_inpatient', 'los_mean_ed', 'los_mean_nonhospitalization',
        'los_min_inpatient', 'los_min_ed', 'los_min_nonhospitalization',
        'los_sum_inpatient', 'los_sum_ed', 'los_sum_nonhospitalization']
    elif len(los.columns) == 8:
        los.columns = ['los_max_inpatient', 'los_max_ed',
        'los_mean_inpatient', 'los_mean_ed',
        'los_min_inpatient', 'los_min_ed', 
        'los_sum_inpatient', 'los_sum_ed']

    
    visits_features = visits_timing.merge(num_visits, how='outer', left_index=True, right_index=True)
    visits_features = visits_features.merge(los, how='outer', left_index=True, right_index=True)
    
    #### VISITS: INPATIENT HOSPITALIZATIONS
    # limit psych hospitalizations to ones eligible (according to preprocessed visits df)
    psych_hospitalizations = temp_visits.loc[temp_visits['visit_occurrence_id'].isin(list_psych_visits)]
    
    # Number of visits
    num_visits = psych_hospitalizations.groupby('person_id').count()['cohort_start_date'].reset_index()
    num_visits.rename({'cohort_start_date':'num_psych_hospitalizations'}, inplace=True, axis=1)
    num_visits.set_index('person_id', inplace=True)
    # adjust so that num_visits is per year of observation
    num_visits = num_visits.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    num_visits = num_visits.div(num_visits.years_obs, axis=0) 
    num_visits.drop(['years_obs'], axis=1, inplace=True)

    visits_features = visits_features.merge(num_visits, how = 'left', right_index=True, left_index=True).fillna(0)

    # Length of Stay
    temp_visits['los'] = (temp_visits['visit_end_date']-temp_visits['visit_start_date']).dt.days
    los = temp_visits.groupby(['person_id']).agg({'los':['sum', 'max', 'min', 'mean']})
    los.columns = [' '.join(col).strip() for col in los.columns.values]
    los.columns = ['los psych sum', 'los psych max', 'los psych min', 'los psych mean']

    visits_features = visits_features.merge(los, how = 'left', right_index=True, left_index=True).fillna(0)

    # Time from most recent visit to end of observation period
    visits_timing = psych_hospitalizations.groupby('person_id').min()['days_to_end_obs']
    visits_timing.name = 'most_recent_psych_inpatient'

    # merge into visits_features
    visits_features = visits_features.merge(visits_timing, how = 'left', right_index=True, left_index=True).fillna(2190)

    ### PROCEDURES
    """
    # drop procedures do not occur in at least 1% of patients
    #procedure_feature_select = temp_procedures[['person_id', 'procedure_concept_id']].drop_duplicates()
    #procedure_feature_select = procedure_feature_select.groupby('procedure_concept_id').count()
    #procedure_feature_select['prevalence'] = 100*procedure_feature_select['person_id']/len(temp_pop)
    #common_procedures = list(procedure_feature_select.loc[procedure_feature_select['prevalence']>1].index)
    
    #all_common_procedures = temp_procedures.loc[temp_procedures['procedure_concept_id'].isin(common_procedures)]
    # use a pivot table to get the counts and binary occurrence of each procedure code
    """
    procedures_features = temp_procedures.pivot_table(index='person_id', columns='concept_name', aggfunc='size', fill_value=0)
    
    # get procedures per year
    procedures_features = procedures_features.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    procedures_features = procedures_features.div(procedures_features.years_obs, axis=0) 
    procedures_features.drop(['years_obs'], axis=1, inplace=True)
    
    ### LABS
    # drop labs that do not occur in at least 1% of patients
    """
    lab_feature_select = temp_labs[['person_id', 'measurement_concept_id']].drop_duplicates()
    lab_feature_select = lab_feature_select.groupby('measurement_concept_id').count()
    lab_feature_select['prevalence'] = 100*lab_feature_select['person_id']/len(temp_pop)
    common_labs = list(lab_feature_select.loc[lab_feature_select['prevalence']>1].index)

    all_common_labs = temp_labs.loc[temp_labs['measurement_concept_id'].isin(common_labs)]
    # use a pivot table to get the counts and binary occurrence of each lab code
    """
    lab_features = temp_labs.pivot_table(index='person_id', columns='concept_name', aggfunc='size', fill_value=0)

    # get procedures per year
    lab_features = lab_features.merge(temp_pop[['person_id','years_obs']].set_index('person_id'), how='left', left_index=True, right_index=True)
    lab_features = lab_features.div(lab_features.years_obs, axis=0) 
    lab_features.drop(['years_obs'], axis=1, inplace=True)
    
    atemporal_features = pd.concat([cond_features, med_features, procedures_features, lab_features, visits_features], axis=1)
    return atemporal_features

### Table 1

In [ ]:
all_race = df_pop.groupby('race_concept_id').count()['cohort_definition_id']
scz_race = df_pop.loc[df_pop['sz_flag']==1].groupby('race_concept_id').count()['cohort_definition_id']
noscz_race = df_pop.loc[df_pop['sz_flag']==0].groupby('race_concept_id').count()['cohort_definition_id']
race_counts = pd.DataFrame(pd.concat([all_race, scz_race, noscz_race], axis=1).values, 
             index=['Missing', 'Black or African American', 'White'], columns = ['All Patients', 'SCZ Patients', 'No SCZ Patients'])

all_gender = df_pop.groupby('gender_concept_id').count()['cohort_definition_id']
scz_gender = df_pop.loc[df_pop['sz_flag']==1].groupby('gender_concept_id').count()['cohort_definition_id']
noscz_gender = df_pop.loc[df_pop['sz_flag']==0].groupby('gender_concept_id').count()['cohort_definition_id']
gender_counts = pd.DataFrame(pd.concat([all_gender, scz_gender, noscz_gender], axis=1).values, 
             index=['Male', 'Female'], columns = ['All Patients', 'SCZ Patients', 'No SCZ Patients'])

age = pd.DataFrame(df_pop.groupby('sz_flag')['age_diagnosis'].agg(['mean','std']).values, index=['SCZ Patients', 'No SCZ Patients'],
            columns = ['Mean Age', 'STD Age']).T
age['All Patients'] = df_pop['age_diagnosis'].mean(), df_pop['age_diagnosis'].std()

t1_counts = pd.concat([race_counts, gender_counts, age])
t1_counts.loc['Total Patients'] = len(df_pop), sum(df_pop['sz_flag']), len(df_pop)-sum(df_pop['sz_flag'])
t1_counts

t1_percents = t1_counts.loc[['Missing', 'Black or African American', 'White', 'Male','Female']]
t1_percents = t1_percents/t1_counts.loc['Total Patients']*100
t1_counts


### Create dataframe that identifies the iteration, first visit (start date), last visit (start date), and years of observation

We do this by ordering all the visits within each patient and then choosing every 5th visit
Then manually want to add back the psychosis date as the first date

In [ ]:
sorted_visits = all_visits.groupby('person_id').apply(pd.DataFrame.sort_values, ['visit_start_date'])
sorted_visits.reset_index(drop=True, inplace=True)
sorted_visits = sorted_visits.merge(df_pop[['person_id', 'psychosis_diagnosis_date', 'first_visit']], how = 'left', left_on = 'person_id', right_on = 'person_id')
sorted_visits = sorted_visits[['person_id', 'psychosis_diagnosis_date', 'first_visit', 'visit_start_date', 'cohort_start_date']].drop_duplicates()
sorted_visits = sorted_visits.loc[(sorted_visits['visit_start_date']>sorted_visits['psychosis_diagnosis_date'])]

In [ ]:
nth_visit = np.asarray(sorted_visits.groupby('person_id').cumcount())
sorted_visits['nth_visit'] = nth_visit

In [ ]:
sorted_visits = sorted_visits.loc[sorted_visits['nth_visit']%6==0]
sorted_visits['nth_visit'] = sorted_visits['nth_visit'] // 6
sorted_visits['nth_visit'] += 1

In [ ]:
add_psychosis_date = df_pop[['person_id', 'psychosis_diagnosis_date', 'first_visit', 'cohort_start_date']]
add_psychosis_date['visit_start_date'] = add_psychosis_date['psychosis_diagnosis_date']
add_psychosis_date['nth_visit'] = 0
df_iter_pop = pd.concat([sorted_visits, add_psychosis_date])

In [ ]:
plt.plot(df_iter_pop['nth_visit'].value_counts())
plt.xlabel('Iteration Number')
plt.ylabel('Number of patients in that iteration')
plt.show()

In [ ]:
df_iter_pop.rename({'nth_visit':'iteration', 'visit_start_date':'cutoff_date'}, axis=1, inplace=True)

df_iter_pop['cohort_start_date'] = pd.to_datetime(df_iter_pop['cohort_start_date'])
df_iter_pop['cutoff_date'] = pd.to_datetime(df_iter_pop['cutoff_date'])
df_iter_pop['first_visit'] = pd.to_datetime(df_iter_pop['first_visit'])

df_iter_pop['years_obs'] = (df_iter_pop['cutoff_date']-df_iter_pop['first_visit']).dt.days/365

In [ ]:
df_iter_pop['censor_date'] = df_iter_pop['cohort_start_date']-pd.Timedelta(90, 'days') 
df_iter_pop = df_iter_pop.loc[df_iter_pop['cutoff_date']<=df_iter_pop['censor_date']]

In [ ]:
df_iter_pop.to_csv('stored_data/iterated_population_6_visits.csv')
print(len(df_iter_pop))

### Loop through iteration to get features for each step

In [ ]:
list_feature_dfs = []

for iteration in tqdm(range(0,df_iter_pop['iteration'].max())): 
    # only need iter0 if you change stuff above
    temp_df_iter_pop = df_iter_pop.loc[(df_iter_pop['iteration'] == iteration)&df_iter_pop['years_obs']>0.5]

    # for conditions, labs, procedures, just compare the start_date to the cutoff date
    temp_conds = all_conds.loc[all_conds['person_id'].isin(temp_df_iter_pop['person_id'])]
    temp_conds = temp_conds.merge(temp_df_iter_pop[['person_id','cutoff_date']], how = 'left', left_on = 'person_id', right_on = 'person_id')
    temp_conds = temp_conds.loc[temp_conds['condition_start_date']< temp_conds['cutoff_date']]

    temp_labs = all_labs.loc[all_labs['person_id'].isin(temp_df_iter_pop['person_id'])]
    temp_labs = temp_labs.merge(temp_df_iter_pop[['person_id','cutoff_date']], how = 'left', left_on = 'person_id', right_on = 'person_id')
    temp_labs = temp_labs.loc[temp_labs['measurement_date']< temp_labs['cutoff_date']]

    temp_procedures = all_procedures.loc[all_procedures['person_id'].isin(temp_df_iter_pop['person_id'])]
    temp_procedures = temp_procedures.merge(temp_df_iter_pop[['person_id','cutoff_date']], how = 'left', left_on = 'person_id', right_on = 'person_id')
    temp_procedures = temp_procedures.loc[temp_procedures['procedure_date']< temp_procedures['cutoff_date']]

    """
    for medications and visits, we want to look at 
    1. limit to visit/medication start dates prior to cutoff date
    2. if cutoff date is prior to visit/medication end date, make the cutoff date the new end date
    """
    temp_meds = all_meds.loc[all_meds['person_id'].isin(temp_df_iter_pop['person_id'])]
    temp_meds = temp_meds.merge(temp_df_iter_pop[['person_id','cutoff_date']], how = 'left', left_on = 'person_id', right_on = 'person_id')
    temp_meds = temp_meds.loc[temp_meds['drug_era_start_date']< temp_meds['cutoff_date']]
    temp_meds.loc[temp_meds['drug_era_end_date']>temp_meds['cutoff_date'], 'drug_era_end_date']=temp_meds['cutoff_date']

    temp_visits = all_visits.loc[all_visits['person_id'].isin(temp_df_iter_pop['person_id'])]
    temp_visits = temp_visits.merge(temp_df_iter_pop[['person_id','cutoff_date']], how = 'left', left_on = 'person_id', right_on = 'person_id')
    temp_visits = temp_visits.loc[temp_visits['visit_start_date']< temp_visits['cutoff_date']]
    temp_visits.loc[temp_visits['visit_end_date']>temp_visits['cutoff_date'], 'visit_end_date']=temp_visits['cutoff_date']
    
    if min((temp_conds['cutoff_date']-temp_conds['condition_start_date']).dt.days) < 0:
        print('Leakage in conds')        
    if min((temp_labs['cutoff_date']-temp_labs['measurement_date']).dt.days) < 0:
        print('Leakage in labs')
    if min((temp_procedures['cutoff_date']-temp_procedures['procedure_date']).dt.days) < 0:
        print('Leakage in procedures')
    if min((temp_visits['cutoff_date']-temp_visits['visit_start_date']).dt.days) < 0:
        print('Leakage in visit starts')
    if min((temp_visits['cutoff_date']-temp_visits['visit_end_date']).dt.days) < 0:
        print('Leakage in visit ends')
    if min((temp_meds['cutoff_date']-temp_meds['drug_era_start_date']).dt.days) < 0:
        print('Leakage in med starts')
    if min((temp_meds['cutoff_date']-temp_meds['drug_era_end_date']).dt.days) < 0:
        print('Leakage in med ends')

    all_features = make_static_df(temp_df_iter_pop, temp_conds, temp_meds, temp_visits, temp_procedures, temp_labs)
    all_features['iteration'] = iteration
    
    all_features.to_csv('stored_data/visit_iters_6/pre_psychosis_features_iter_'+str(iteration)+'.csv')


### Read in and concatenate the dataframes

In [5]:
list_files = []
list_filenames = os.listdir('stored_data/visit_iters_6')
for filename_ind in tqdm(range(len(list_filenames))):
    filename = list_filenames[filename_ind]
    list_files.append(pd.read_csv('stored_data/visit_iters_6/'+filename))

100%|██████████████████████████████████████████████████| 590/590 [04:55<00:00,  1.99it/s]


In [6]:
df_all_iters = pd.concat(list_files)
df_all_iters.fillna(0, inplace=True)

In [7]:
print('Unnamed: 0' in df_all_iters.columns, 'Unnamed:0' in df_all_iters.columns)

False False


In [8]:
del list_files
gc.collect()

0

# Training the XGBoost Model

### Train test split

In [9]:
num_days_prediction = 90
df_pop = pd.read_csv(path+'population.csv')
df_pop.rename({'psychosis_dx_date':'psychosis_diagnosis_date'}, axis=1, inplace=True)
df_pop['psychosis_diagnosis_date'] = pd.to_datetime(df_pop['psychosis_diagnosis_date'], format="mixed")
df_pop['cohort_start_date'] = pd.to_datetime(df_pop['cohort_start_date'])
df_pop = df_pop.loc[(df_pop['cohort_start_date']-df_pop['psychosis_diagnosis_date']).dt.days >= num_days_prediction]

In [10]:
# ONLY IF YOU ARE RESTRICTING THE DATA IN SOME WAY

"""
#restricting to some number of iterations
df_all_iters = df_all_iters.loc[df_all_iters['iteration']<=209]
df_pop = df_pop.loc[df_pop['person_id'].isin(df_all_iters['person_id'])]
"""
"""
### restricting to only data points where we have 7 years of data

df_iter_pop = pd.read_csv('stored_data/iterated_population_6_visits.csv')

df_iter_pop['cutoff_date'] = pd.to_datetime(df_iter_pop['cutoff_date'])
df_iter_pop['psychosis_diagnosis_date'] = pd.to_datetime(df_iter_pop['psychosis_diagnosis_date'])
df_iter_pop['years_since_psychosis_dx'] = (df_iter_pop['cutoff_date']-df_iter_pop['psychosis_diagnosis_date']).dt.days/365
df_iter_pop = df_iter_pop.loc[df_iter_pop['years_since_psychosis_dx']<=7]

df_all_iters.set_index(['person_id', 'iteration'], inplace=True)

arr_vals = df_iter_pop[['person_id', 'iteration']].values
pid_iter = [tuple(x) for x in arr_vals]

df_all_iters = df_all_iters.loc[df_all_iters.index.isin(pid_iter)]
df_all_iters.reset_index(inplace=True)
df_pop = df_pop.loc[df_pop['person_id'].isin(df_all_iters['person_id'])]


### restricting to only people who have <= 7 years of data between psychosis and observation
df_iter_pop = pd.read_csv('stored_data/iterated_population_6_visits.csv')

df_iter_pop['cutoff_date'] = pd.to_datetime(df_iter_pop['cutoff_date'])
df_iter_pop['psychosis_diagnosis_date'] = pd.to_datetime(df_iter_pop['psychosis_diagnosis_date'])
df_iter_pop['years_since_psychosis_dx'] = (df_iter_pop['cutoff_date']-df_iter_pop['psychosis_diagnosis_date']).dt.days/365
max_obs = df_iter_pop.groupby('person_id').max()['years_since_psychosis_dx']

patient_list = list(max_obs.loc[max_obs<=7].index)
df_all_iters = df_all_iters.loc[df_all_iters['person_id'].isin(patient_list)]
df_pop = df_pop.loc[df_pop['person_id'].isin(df_all_iters['person_id'])]

"""

In [11]:
labels = df_all_iters[['person_id', 'iteration']].merge(df_pop[['person_id','sz_flag']], how='left', left_on = 'person_id', right_on='person_id')
labels.set_index('person_id', inplace=True)

df_all_iters.set_index('person_id', inplace=True)
df_all_iters.drop(['iteration'], inplace=True, axis=1)

In [12]:
labels.isna().sum()

iteration    0
sz_flag      0
dtype: int64

In [13]:
pid_train, pid_test, y_train, y_test = train_test_split(df_pop['person_id'], df_pop['sz_flag'], stratify=df_pop['sz_flag'], test_size=0.3, random_state = 4)

X_train = df_all_iters.loc[pid_train.values]
X_test = df_all_iters.loc[pid_test.values]

y_train = labels.loc[pid_train.values, 'sz_flag']
y_test = labels.loc[pid_test.values, 'sz_flag']

In [14]:
save_cols = df_all_iters.columns

In [15]:
del df_all_iters
del labels
gc.collect()

0

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:507: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:507: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/usr/lib/python3/dist-packages/sklearn/utils/validation.py:507: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


### Fit XGBoost Classifier

In [17]:
max_depths = np.asarray([3,4,5])
estimators = np.asarray([150,200,250,300])
params = list(product(max_depths, estimators))

list_aucs = []

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=3)
for p in params:
    print(p)
    depth, est = p
    param_aucs = []
    for i, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
        train_temp, val_temp = X_train[train_index,:], X_train[val_index]
        y_train_temp, y_val_temp = y_train.iloc[train_index], y_train.iloc[val_index]
        
        clf = XGBClassifier(seed=3, max_depth = depth, n_estimators = est)
        clf.fit(train_temp, y_train_temp)
        y_pred_temp = clf.predict(val_temp)
        param_aucs.append(roc_auc_score(y_val_temp, y_pred_temp))
        
        del train_temp
        del val_temp
        gc.collect()
    list_aucs.append(param_aucs)
    print(np.mean(param_aucs))

(3, 150)


/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning

0.9685988661047122
(3, 200)


/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning

0.9782467080326438
(3, 250)


...

0.9936492290997687
(5, 250)


/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning

0.9951481578915796
(5, 300)


/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning

0.9959400794977056


In [18]:
for i, j in zip(params, list_aucs):
    print(i, np.mean(j))

(3, 150) 0.9685988661047122
(3, 200) 0.9782467080326438
(3, 250) 0.9836718692655071
(3, 300) 0.9868792923824653
(4, 150) 0.9845632869392483
(4, 200) 0.9894929909001565
(4, 250) 0.992253728583081
(4, 300) 0.9938729835946502
(5, 150) 0.9912097458933499
(5, 200) 0.9936492290997687
(5, 250) 0.9951481578915796
(5, 300) 0.9959400794977056


In [19]:
clf = XGBClassifier(seed=3, max_depth = 5, n_estimators = 300)
clf.fit(X_train, y_train)

with open('models/xgb_6_visits_7yr_patient_cutoff.pkl','wb') as f:
    pickle.dump(clf,f)

/data/home/ak4885/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [ ]:
#y_test

y_pred = clf.predict(X_test)
roc_auc_score(y_test, y_pred)